In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract product title
def get_title(soup):
    
    try:
        #outer Tag Object
        title = soup.find("span",attrs={"id":'productTitle'})
        
        #inner NavigatableString Object
        title_value = title.text
        
        #title as a string value
        title_string = title_value.strip()
        
    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price-whole'}).text

        except:
            price = ""

    return price


# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = "" 

    return rating
       

In [3]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=watches+for+women&crid=SXD586EFNWRO&sprefix=watches%2Caps%2C355&ref=nb_sb_ss_ts-doa-p_1_7"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[] }
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in/" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [4]:
amazon_df

,title,price,rating
2,Noise ColorFit Pulse Grand Smart Watch with 1....,"1,999.",4.0 out of 5 stars
11,New Fastrack Reflex Beat+ 1.69” UltraVU Displa...,"1,495",4.0 out of 5 stars
15,Yellow Chimes Stainless-Steel Base Metal Pink ...,245.,4.1 out of 5 stars
40,Fastrack Trendies Analog Pink Dial Women's Wat...,"1,975.",4.3 out of 5 stars
50,LOUIS DEVIN Rose Gold Plated Mesh Chain Analog...,359,3.9 out of 5 stars
60,SELLORIA Analouge White Dial Multi Color Strap...,249.,3.2 out of 5 stars
